## ICE Summer School : SN Cosmology
 * Notebook developed by Mathew Smith (mat.smith@lancaster.ac.uk)

## Part 2: Measuring the cosmological parameters

 * In Part 1 we determined the distance to SNeIa in the local universe
 * Today, we will exploit this information to measure the cosmological parameters
 * The state-of-the-art at high redshift comes from the Dark Energy Survey (DES-5YR). 
 * We will download and exploit this dataset.
 * We will use the distances derived to these SNe to measure the cosmological parameters    

Required libraries: 
 * numpy
 * pandas
 * scipy
 * matplotlib
 * astropy
 * scipy
 * emcee
 * corner
 * chainconsumer

In [1]:
import numpy as np
import pandas as pd
from scipy import optimize as opt
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from astropy.cosmology import Planck18, FlatLambdaCDM, LambdaCDM, FlatwCDM, Flatw0waCDM
import emcee
import corner
from chainconsumer import Chain, ChainConsumer, make_sample, Truth, PlotConfig

Some simple shortcuts for plotting and calculations

In [2]:
fac       = 2.5/np.log(10)
dummy_z   = np.linspace(0,1.2,100)
#
colors    = {'blue':'#194D80', 'yellow':'#F8AD05', 'silver':'#ADADAD', 'green':'#566E3D', 'red':'#A22522'}

## The data

The latest results from DES can be downloaded from:
 * `https://raw.githubusercontent.com/des-science/DES-SN5YR/main/4_DISTANCES_COVMAT/DES-SN5YR_HD+MetaData.csv`. 

This dataset contains 1829 SNeIa each with a measured redshift (`zHD`), distance (`MU`) and uncertainty (`MUERR_FINAL`). 

Distances were derived using: 
 * $\alpha = 0.16087 \pm 0.00152$
 * $\beta = 3.11780 \pm 0.03530$
 * $\gamma = 0.03754 \pm 0.00798$
 * $M0_{avg} = −29.95821$

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/des-science/DES-SN5YR/main/4_DISTANCES_COVMAT/DES-SN5YR_HD+MetaData.csv", sep=',')
df.head()

,CID,CIDint,IDSURVEY,TYPE,zHEL,zHELERR,zCMB,zCMBERR,zHD,zHDERR,...,PROB_SNIRFV19,PROB_SNNDESCC,PROB_SNNJ17,PROB_SNNV19,MU,MUERR_FINAL,PROBCC_BEAMS,biasCor_mu,biasCor_muCOVSCALE,biasCor_muCOVADD
0,1246275,1246275,10,0,0.24651,0.001,0.24605,0.001,0.24605,0.00160,...,0.8486,1.0000,0.9999,1.0000,40.5938,0.0968,0.0,0.0341,1.0,0.0050
1,1246281,1246281,10,0,0.33600,0.001,0.33549,0.001,0.33549,0.00167,...,1.0000,1.0000,0.9999,1.0000,41.2263,0.1360,0.0,-0.0492,1.0,0.0136
2,1246314,1246314,10,0,0.38388,0.001,0.38337,0.001,0.38337,0.00171,...,0.7823,0.9993,0.9970,0.9998,41.6383,0.2332,0.0,0.0502,1.0,0.0348
3,1246527,1246527,10,0,0.32184,0.001,0.32078,0.001,0.32078,0.00166,...,1.0000,0.9997,0.9998,1.0000,41.1991,0.1503,0.0,-0.0511,1.0,0.0173
4,1246529,1246529,10,0,0.49797,0.001,0.49677,0.001,0.49677,0.00180,...,0.9407,0.9996,0.9993,1.0000,42.1471,0.1618,0.0,-0.0485,1.0,0.0095


For this exercise, remove events with low probability of being an SNIa

In [4]:
df         = df[df.PROB_SNNDESCC>0.9]

### Exercise: Make a Hubble Diagram
   * Given the dataset, make a plot of redshift ($zHD$) v distance modulus ($\mu$) including uncertainties
   * Using the astropy.cosmology module, plot the residuals from a FlatLCDM model
   * Also include the residuals from a FlatwCDM and Flatw0waCDM model with parameter values taken from DES 2025 (https://arxiv.org/pdf/2401.02929)

***

## Comparing our data to a cosmological model
 * To do this we will use an MCMC algorithm (`emcee`)
 * This requires a function to determine the goodness of fit between the data and model
 * We will 'minimise the posterior'; this is an extension of a traditional $\chi^2$ minimisation

 * The aim is to minimise the `logposterior`:
     * This is the sum of two terms : `loglikelihood` and `logprior`
     * i.e. $\ln \mathcal{P}(\theta | x,y) = \ln \mathcal{L}(\theta | x,y) + \ln \text{prior}(\theta)$
     * where $\theta$ are the model parameters being tested, $(x,y)$ is the data we have collected
 * The `logprior` encodes our knowledge of the _prior_ to collecting the data
     * It is therefore independent of the data
     * An assumption of a `uniform` (or flat) prior is often taken:
        * i.e. all values in parameter space (within a given bound) are equally likely to be true
     * We will assume that `(0.25 < $\Omega_m$ < 0.45)` & `(65. < $H_0$ < 75.)`
 * The `loglikelihood` measures the goodness of fit between the model and our dataset
     * $\ln \mathcal{L}(\theta | x,y) = -0.5 \times \left( N\times\ln(2\pi) + \sum_{i=0}^{N}2\times\ln(\sigma_i) + \sum_{i=0}^{N} \left(\frac{y - f(y,*\theta)}{\sigma_i}\right)^2\right)$
     * _Note_: the final term in this equation is the traditional $\chi^2$ formula


### Exercise: 
 * Define the loglikelihood, logprior and logposterior between the DES5YR dataset and a FlatLCDM model
     * _Hint_ : If needed likelihood can be simplified to the $\chi^2$

***

## Estimating the cosmological parameters
 * To do this we will use the `emcee` module: combining our measurements with the posterior code

## Exercise: 
 * Use the `emcee` library to produce an MCMC chain for this problem.
    * _NB_: 50 walkers, each with 500 steps will be sufficient
 * Use the `corner` or `chainconsumer` library to visualise the results
 * Highlight the best-fit value
 * _Extension_ : Consider the `LambdaCDM` or `FlatwCDM` models

***